In [1]:
! pip install mip

In [2]:
from mip import *

In [ ]:
EPS = 10e-4

def cg_2():
    """
    Implementação simples de geração de colunas (para fins didáticos)
    """

    L = 250                  # tamanho de cada barra
    m = 4                    # numero de pedidos
    w = [ 187, 119, 74, 90 ] # tamanho de cada pedido i
    b = [ 1, 2, 2, 1 ]       # demanda de cada pedido i

    # criando os modelos e estruturas de dados
    master = Model()
    lambdas = []
    constraints = []

    # criando primeiros padrões (que cortam exatamente um item por barra)
    # o objetivo é iniciar o mestre restrito com uma solução factível
    for i in range(m):
        lambdas.append(master.add_var(obj=1, name='lambda_%d' % (len(lambdas)+1)))

    # criando restrições (vazias pois nao ha variaveis neste momento)
    for i in range(m):
        constraints.append(master.add_constr(lambdas[i] >= b[i], name='i_%d' % (i+1)))

    # hora de criar o problema de pricing...
    pricing = Model()

    # criando variaveis do pricing
    a = []
    for i in range(m):
        a.append(pricing.add_var(obj=0, var_type='I', name='a_%d' % (i+1)))

    # criando a unica restricao do pricing
    pricing.add_constr(sum(w[i] * a[i] for i in range(m)) <= L, name='tamanho_barra')

    #pricing.write('pricing.lp')

    new_vars = True
    while (new_vars):

        ##########
        # PASSO 1: resolvendo o mestre
        ##########

        master.optimize()
        master.write('master.lp')

        ##########
        # PASSO 2: atualizando o pricing com os valores duais do mestre
        ##########

        # atualizando a função objetivo do pricing
        pricing.objective = 1
        for i in range(m):
            a[i].obj = -constraints[i].pi

        pricing.write('pricing.lp')

        # resolvendo o pricing
        pricing.optimize()

        print_solution(master)
        print('pi = ', end='')
        print([ constraints[i].pi for i in range(m) ])
        print('')

        print('Pricing:')
        print('    z =  {pricing.objective_value}'.format(**locals()))
        print('    a = ', end='')
        print([ v.x for v in pricing.vars ])
        print('')

        ##########
        # PASSO 3: inserindo as novas colunas (caso alguma exista)
        ##########

        # checando se foi encontrada alguma variável com custo reduzido negativo
        # e inserindo no mestre em caso positivo
        if pricing.objective_value < - EPS:
            coeffs = [ a[i].x for i in range(m) ]
            column = Column(constraints, coeffs)
            lambdas.append(master.add_var(obj=1, column=column, name='lambda_%d' % (len(lambdas)+1)))

            print('novo padrao = {coeffs}'.format(**locals()))

        # se não foi encontrada nenhuma variável atrativa, o problema foi resolvido!
        else:
            new_vars = False

        pricing.write('pricing.lp')

    print_solution(master)

In [ ]:
EPS = 10e-4

def cg(L, m, w, b):
    """
    Implementação simples de geração de colunas (para fins didáticos)
    """

    # criando os modelos e estruturas de dados
    master = Model("Cutting Stock: Geração de Colunas: Mestre")
    lambdas = []
    constraints = []

    # criando primeiros padrões (que cortam exatamente um item por barra)
    # o objetivo é iniciar o mestre restrito com uma solução factível
    for i in range(m):
        lambdas.append(master.add_var(obj=1, name='lambda_%d' % (len(lambdas)+1)))

    # criando restrições (vazias pois nao ha variaveis neste momento)
    for i in range(m):
        constraints.append(master.add_constr(lambdas[i] >= b[i], name='i_%d' % (i+1)))

    # hora de criar o problema de pricing...
    pricing = Model("Cutting Stock: Geração de Colunas: Pricing")

    # criando variaveis do pricing
    a = []
    for i in range(m):
        a.append(pricing.add_var(obj=0, var_type='I', name='a_%d' % (i+1)))

    # criando a unica restricao do pricing
    pricing.add_constr(sum(w[i] * a[i] for i in range(m)) <= L, name='tamanho_barra')

    #pricing.write('pricing.lp')

    new_vars = True
    while (new_vars):

        ##########
        # PASSO 1: resolvendo o mestre
        ##########

        master.optimize()
        master.write('master.lp')

        ##########
        # PASSO 2: atualizando o pricing com os valores duais do mestre
        ##########

        # atualizando a função objetivo do pricing
        pricing.objective = 1
        for i in range(m):
            a[i].obj = -constraints[i].pi

        pricing.write('pricing.lp')

        # resolvendo o pricing
        pricing.optimize()

        print_solution(master)
        print('pi = ', end='')
        print([ constraints[i].pi for i in range(m) ])
        print('')

        print('Pricing:')
        print('    z =  {pricing.objective_value}'.format(**locals()))
        print('    a = ', end='')
        print([ v.x for v in pricing.vars ])
        print('')

        ##########
        # PASSO 3: inserindo as novas colunas (caso alguma exista)
        ##########

        # checando se foi encontrada alguma variável com custo reduzido negativo
        # e inserindo no mestre em caso positivo
        if pricing.objective_value < - EPS:
            coeffs = [ a[i].x for i in range(m) ]
            column = Column(constraints, coeffs)
            lambdas.append(master.add_var(obj=1, column=column, name='lambda_%d' % (len(lambdas)+1)))

            print('novo padrao = {coeffs}'.format(**locals()))

        # se não foi encontrada nenhuma variável atrativa, o problema foi resolvido!
        else:
            new_vars = False

        pricing.write('pricing.lp')

    print_solution(master)

In [ ]:
def kantorovich(N, L, m, w, b):
    """
    Implementacao do modelo de Kantorovich para o exemplo da aula
    """

    # criando os modelos e estruturas de dados
    model = Model("Cutting Stock: Kantorovich")
    x = {(i, j): model.add_var(obj=0, var_type=INTEGER, name="x[%d,%d]" % (i, j))
        for i in range(m) for j in range(N)}
    y = {j: model.add_var(obj=1, var_type=BINARY, name="y[%d]" % j)
        for j in range(N)}

    # restricoes
    for i in range(m):
        model.add_constr(xsum(x[i, j] for j in range(N)) >= b[i])
    for j in range(N):
        model.add_constr(xsum(w[i] * x[i, j] for i in range(m)) <= L * y[j])

    # resolvendo o problema
    model.optimize()
    print_solution(model)

    # resolvendo a relaxacao linear
    model.relax()
    model.optimize()
    print_solution(model)

def print_solution(model):
    print('')
    print('Custo:\n    {model.objective_value:.3f}'.format(**locals()))
    print('Solucao do %s:' % model.name)
    for v in model.vars:
        if v.x > EPS:
            print('    {v.name} = {v.x:.3f}'.format(**locals()))


In [ ]:
def solve_problem(N, L, m, w, b):
    print("=== Geração de Colunas ===")
    cg(L, m, w, b)

    print("\n=== Kantorovich ===")
    kantorovich(N, L, m, w, b)


In [ ]:
if __name__ == "__main__":
    # Exemplo 1:
    N = 10                   # numero maximo de barras
    L = 250                  # tamanho de cada barra
    m = 4                    # numero de pedidos
    w = [ 187, 119, 74, 90 ] # tamanho de cada pedido i
    b = [ 1, 2, 2, 1 ]       # demanda de cada pedido i

    print("> Exemplo #1 <")
    solve_problem(N, L, m, w, b)

    # Exemplo 2:
    N = 500                  # numero maximo de barras
    L = 100                  # tamanho de cada barra
    m = 4                    # numero de pedidos
    w = [ 14, 31, 36, 45 ] # tamanho de cada pedido i
    b = [ 211, 395, 610, 97 ]       # demanda de cada pedido i

    print("\n> Exemplo #2 <")
    solve_problem(N, L, m, w, b)
